In [1]:
from sklearn.datasets import fetch_openml
from sklearn import datasets
import time
import numpy as np
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn import metrics
import statistics
from sklearn.svm import SVC

In [2]:
#Data Set Up

In [3]:
extract = np.asarray(pd.read_fwf(os.getcwd()+"/Downloads/Ecoli/ecoli.data"))

In [4]:
extract1 = extract[:,1:]
extract1
X = extract1[:,:7]
Y = extract1[:,7]
print(X)
print(Y)

[[0.07 0.4 0.48 ... 0.54 0.35 0.44]
 [0.56 0.4 0.48 ... 0.49 0.37 0.46]
 [0.59 0.49 0.48 ... 0.52 0.45 0.36]
 ...
 [0.61 0.6 0.48 ... 0.44 0.39 0.38]
 [0.59 0.61 0.48 ... 0.42 0.42 0.37]
 [0.74 0.74 0.48 ... 0.31 0.53 0.52]]
['cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'im' 'im' 'im' 'im' 'im' 'im' 'im' 'im' 'im' 'im' 'im'

In [5]:
m = X.shape[0]
n = X.shape[1] + 1
X = np.concatenate((np.ones([m,1]),X), axis=1) # add one to each data point
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.15, random_state=46)

In [6]:
print('information of datasets:')
print(f'- X original: {X.shape[0]} x {X.shape[1]}')
print(Y.shape)

information of datasets:
- X original: 335 x 8
(335,)


In [7]:
#Binary Classification without kernels

In [8]:
cp = "cp"
im = "im"

whereiscp = np.where(Y==cp)[0]
whereisim = np.where(Y==im)[0]

indxcpim = np.concatenate((whereiscp,whereisim))

Xcpim = X[indxcpim]
Ycpim = np.ones(len(indxcpim))
Ycpim[whereisim] =-1

xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xcpim, Ycpim, test_size = 0.30, random_state=46)

In [9]:
#Creating Classifier
N = len(ydual_train)
XPY = xdual_train.copy()
for i in range(N):
    if ydual_train[i]==-1:
        XPY[i,:] =-1 * XPY[i,:]   

A = np.matmul(XPY,XPY.transpose())

YM = np.outer(ydual_train[1:],ydual_train[1:])
AY = np.outer(A[0,1:],ydual_train[1:])
YA = np.outer(ydual_train[1:],A[0,1:])


Y0S = ydual_train[0]**2
M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
aw = np.zeros(N)

bright = np.zeros(N)
bright = 1 - ydual_train/ydual_train[0]

adiag = np.diagonal(A)
t1    = A[0,0]
t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
t3    = adiag + t1 - t2
indxnzero = np.where(t3 != 0)
aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
YA = np.zeros(N)
YA[0] = ydual_train[0]*aw[0]
YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
wght = sum(xdual_train * YA[:,None])
b =sum(ydual_train - np.matmul(xdual_train,wght))/N

In [10]:
#Creating fit
yfit = np.matmul(xdual_test,wght) + b
yfitsign = np.sign(yfit)
ydiff = yfitsign - np.asfarray(ydual_test)
wherediff = np.where(ydiff != 0)

In [11]:
print("Accuracy = ", 1- len(wherediff[0])/len(ydual_test))
print("Error = ", len(wherediff[0])/len(ydual_test))

Accuracy =  0.9393939393939394
Error =  0.06060606060606061


In [12]:
#Binary Sklearn without kernels

In [13]:
clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(xdual_train, ydual_train)
y_pred = clf.predict(xdual_test)

In [14]:
print("Accuracy:",metrics.accuracy_score(ydual_test, y_pred))
print("Error:",1 - metrics.accuracy_score(ydual_test, y_pred))

Accuracy: 0.9848484848484849
Error: 0.015151515151515138


In [15]:
#SUMMARY OF BINARY WITHOUT KERNELS

In [16]:
#It looks like the errors and accuracy of our own SVM are similar to the accuarcy and errors of the SKlearn binary
#SVM. We are off by about 5% for accuracy and error comparing each of these models.

In [17]:
#Binary Classification with kernels

In [18]:
#Polynomial

In [19]:
cp = "cp"
im = "im"

whereiscp = np.where(Y==cp)[0]
whereisim = np.where(Y==im)[0]

indxcpim = np.concatenate((whereiscp,whereisim))

Xcpim = X[indxcpim]
Ycpim = np.ones(len(indxcpim))
Ycpim[whereisim] =-1

xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xcpim, Ycpim, test_size = 0.30, random_state=46)

In [20]:
#Creating Classifier
N = len(ydual_train)
XPY = xdual_train.copy()
for i in range(N):
    if ydual_train[i]==-1:
        XPY[i,:] =-1 * XPY[i,:]   

A = (np.matmul(XPY,XPY.transpose())+1)**2

YM = np.outer(ydual_train[1:],ydual_train[1:])
AY = np.outer(A[0,1:],ydual_train[1:])
YA = np.outer(ydual_train[1:],A[0,1:])


Y0S = ydual_train[0]**2
M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
aw = np.zeros(N)

bright = np.zeros(N)
bright = 1 - ydual_train/ydual_train[0]

adiag = np.diagonal(A)
t1    = A[0,0]
t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
t3    = adiag + t1 - t2
indxnzero = np.where(t3 != 0)
aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
YA = np.zeros(N)
YA[0] = ydual_train[0]*aw[0]
YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
wght = sum(xdual_train * YA[:,None])
b =sum(ydual_train - np.matmul(xdual_train,wght))/N

In [21]:
#Creating fit
yfit = np.matmul(xdual_test,wght) + b
yfitsign = np.sign(yfit)
ydiff = yfitsign - np.asfarray(ydual_test)
wherediff = np.where(ydiff != 0)

In [22]:
print("Accuracy = ", 1- len(wherediff[0])/len(ydual_test))
print("Error = ", len(wherediff[0])/len(ydual_test))

Accuracy =  0.9393939393939394
Error =  0.06060606060606061


In [23]:
#Polynomial sklearn

In [24]:
clf = svm.SVC(kernel='poly')
clf.fit(xdual_train,ydual_train)
y_pred = clf.predict(xdual_test)

In [25]:
print("Accuracy:",metrics.accuracy_score(ydual_test, y_pred))
print("Error:",1 - metrics.accuracy_score(ydual_test, y_pred))

Accuracy: 0.9848484848484849
Error: 0.015151515151515138


In [26]:
#SUMMARY OF BINARY WITH POLYNOMIAL KERNEL

In [27]:
#It looks like our own binary SVM with a polynomial kernel has similar Accuracy and Error as the SKlearn polynomial
#SVM. As we can see our own accuracy stayed the same with the addition of the polynomial, and the skleanr accuracy  
#also stayed the same as before.

In [28]:
#Radial

In [29]:
cp = "cp"
im = "im"

whereiscp = np.where(Y==cp)[0]
whereisim = np.where(Y==im)[0]

indxcpim = np.concatenate((whereiscp,whereisim))

Xcpim = X[indxcpim]
Ycpim = np.ones(len(indxcpim))
Ycpim[whereisim] =-1

xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xcpim, Ycpim, test_size = 0.30, random_state=46)

In [30]:
#Creating Classifier
N = len(ydual_train)
XPY = xdual_train.copy()
for i in range(N):
    if ydual_train[i]==-1:
        XPY[i,:] =-1 * XPY[i,:]   

A = np.exp(-(LA.norm(XPY-XPY)**2)/np.var(XPY))

#YM = np.outer(ydual_train[1:],ydual_train[1:])
#AY = np.outer(A[0,1:],ydual_train[1:])
#YA = np.outer(ydual_train[1:],A[0,1:])


#Y0S = ydual_train[0]**2
#M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
aw = np.zeros(N)

bright = np.zeros(N)
bright = 1 - ydual_train/ydual_train[0]

adiag = A
t1    = A
t2    = 2*np.multiply(A,ydual_train)/ydual_train[0]
t3    = adiag + t1 - t2
indxnzero = np.where(t3 != 0)
aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
YA = np.zeros(N)
YA[0] = ydual_train[0]*aw[0]
YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
wght = sum(xdual_train * YA[:,None])
b =sum(ydual_train - np.matmul(xdual_train,wght))/N

In [31]:
#Creating fit
yfit = np.matmul(xdual_test,wght) + b
yfitsign = np.sign(yfit)
ydiff = yfitsign - np.asfarray(ydual_test)
wherediff = np.where(ydiff != 0)

In [32]:
print("Accuracy = ", 1- len(wherediff[0])/len(ydual_test))
print("Error = ", len(wherediff[0])/len(ydual_test))

Accuracy =  0.9545454545454546
Error =  0.045454545454545456


In [33]:
#Radial Sklearn

In [34]:
clf = svm.SVC(kernel='rbf')
clf.fit(xdual_train,ydual_train)
y_pred = clf.predict(xdual_test)

In [35]:
print("Accuracy:",metrics.accuracy_score(ydual_test, y_pred))
print("Error:",1 - metrics.accuracy_score(ydual_test, y_pred))

Accuracy: 1.0
Error: 0.0


In [36]:
#SUMMARY OF BINARY WITH RADIAL KERNEL

In [37]:
# We again have very similar Accuracy and Error rates between sklearn and our own version of Binary with a radial
#kernel. For our own version we see that the accuracy increased with the addition of the radial kernel, 
# The accuracy of the radial sklearn model alo increased which it ended up having a perfect accuracy.

In [38]:
#Multiclass Classification without kernels

In [39]:
results = np.array([])

In [40]:
#CP vs all

In [41]:
def cpvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=46)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.matmul(XPY,XPY.transpose())

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [42]:
cpwherediff, cpwght, cpydual_test, cpb = cpvsall(X,Y)

In [43]:
print(len(cpwherediff)/len(cpydual_test))

0.1568627450980392


In [44]:
#creating fit on main test for cp

In [45]:
cpx_test = x_test[np.where(y_test == "cp")[0]]
cpy_test = y_test[np.where(y_test == "cp")[0]]

In [46]:
cpyfit = np.matmul(cpx_test,cpwght) + cpb
cpyfitsign = np.sign(cpyfit)
print(cpyfitsign)
print(cpy_test)

[1 1 -1 1 1 -1 -1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
['cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp']


In [47]:
results = np.append(results,cpyfitsign)

In [48]:
#im vs all

In [49]:
def imvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=46)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.matmul(XPY,XPY.transpose())

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [50]:
imwherediff, imwght, imydual_test, imb = imvsall(X,Y)

In [51]:
print(len(imwherediff)/len(imydual_test))

0.19607843137254902


In [52]:
#creating fit on main test for im

In [53]:
imx_test = x_test[np.where(y_test == "im")[0]]
imy_test = y_test[np.where(y_test == "im")[0]]

In [54]:
imyfit = np.matmul(imx_test,imwght) + imb
imyfitsign = np.sign(imyfit)
print(imyfitsign)
print(imy_test)

[1 1 1 1 -1 1 1 1]
['im' 'im' 'im' 'im' 'im' 'im' 'im' 'im']


In [55]:
results = np.append(results,imyfitsign)

In [56]:
#ms va all

In [57]:
def msvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=46)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.matmul(XPY,XPY.transpose())

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [58]:
mswherediff, mswght, msydual_test, msb = msvsall(X,Y)

In [59]:
print(len(mswherediff)/len(msydual_test))

0.19607843137254902


In [60]:
#creating fit on main test for ms

In [61]:
msx_test = x_test[np.where(y_test == "mS")[0]]
msy_test = y_test[np.where(y_test == "mS")[0]]

In [62]:
msyfit = np.matmul(msx_test,mswght) + msb
msyfitsign = np.sign(msyfit)
print(msyfitsign)
print(msy_test)

[]
[]


In [63]:
results = np.append(results,msyfitsign)

In [64]:
#ml vs all

In [65]:
def mlvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=46)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.matmul(XPY,XPY.transpose())

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [66]:
mlwherediff, mlwght, mlydual_test, mlb = mlvsall(X,Y)

In [67]:
print(len(mlwherediff)/len(mlydual_test))

0.3137254901960784


In [68]:
#creating fit on main test for ml

In [69]:
mlx_test = x_test[np.where(y_test == "mL")[0]]
mly_test = y_test[np.where(y_test == "mL")[0]]

In [70]:
mlyfit = np.matmul(mlx_test,mlwght) + mlb
mlyfitsign = np.sign(mlyfit)
print(mlyfitsign)
print(mly_test)

[1]
['mL']


In [71]:
results = np.append(results,mlyfitsign)

In [72]:
#mu vs all

In [73]:
def muvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=46)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.matmul(XPY,XPY.transpose())

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [74]:
muwherediff, muwght, muydual_test, mub = muvsall(X,Y)

In [75]:
print(len(muwherediff)/len(muydual_test))

0.23529411764705882


In [76]:
#creating fit on main test for mu

In [77]:
mux_test = x_test[np.where(y_test == "mU")[0]]
muy_test = y_test[np.where(y_test == "mU")[0]]

In [78]:
muyfit = np.matmul(mux_test,muwght) + mub
muyfitsign = np.sign(muyfit)
print(muyfitsign)
print(muy_test)

[1 1 1 1 1]
['mU' 'mU' 'mU' 'mU' 'mU']


In [79]:
results = np.append(results,muyfitsign)

In [80]:
#om vs all

In [81]:
def omvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=46)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.matmul(XPY,XPY.transpose())

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [82]:
omwherediff, omwght, omydual_test, omb = omvsall(X,Y)

In [83]:
print(len(omwherediff)/len(omydual_test))

0.35294117647058826


In [84]:
#creating fit on main test for om

In [85]:
omx_test = x_test[np.where(y_test == "om")[0]]
omy_test = y_test[np.where(y_test == "om")[0]]

In [86]:
omyfit = np.matmul(omx_test,omwght) + omb
omyfitsign = np.sign(omyfit)
print(omyfitsign)
print(omy_test)

[1 1]
['om' 'om']


In [87]:
results = np.append(results,omyfitsign)

In [88]:
# pp vs all

In [89]:
def ppvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=46)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.matmul(XPY,XPY.transpose())

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [90]:
ppwherediff, ppwght, ppydual_test, ppb= ppvsall(X,Y)

In [91]:
print(len(ppwherediff)/len(ppydual_test))

0.3333333333333333


In [92]:
#creating fit on main test for pp

In [93]:
ppx_test = x_test[np.where(y_test == "pp")[0]]
ppy_test = y_test[np.where(y_test == "pp")[0]]

In [94]:
ppyfit = np.matmul(ppx_test,ppwght) + ppb
ppyfitsign = np.sign(ppyfit)
print(ppyfitsign)
print(ppy_test)

[1 1 1 -1 1 -1]
['pp' 'pp' 'pp' 'pp' 'pp' 'pp']


In [95]:
results = np.append(results,ppyfitsign)
results.size

51

In [96]:
Right = results[results == 1].size
Wrong = results[results == -1].size
Total = results.size
print("Accuracy = ", Right/Total)
print("Error = ", Wrong/Total)

Accuracy =  0.8823529411764706
Error =  0.11764705882352941


In [97]:
#Multiclass classification with no kernels Sklearn

In [98]:
m = OneVsRestClassifier(LinearSVC())
m.fit(x_train,y_train)

/Users/aidenkempen/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/aidenkempen/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/aidenkempen/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/aidenkempen/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


OneVsRestClassifier(estimator=LinearSVC())

In [99]:
m.classes_ , m.multilabel_

(array(['cp', 'im', 'mL', 'mS', 'mU', 'om', 'pp'], dtype='<U2'), False)

In [100]:
print("Accuracy = ", m.score(x_test,y_test) )
print("Error = ", 1 - m.score(x_test,y_test))

Accuracy =  0.8823529411764706
Error =  0.11764705882352944


In [101]:
#Summary of Multiclass classification with no kernels

In [102]:
# We see that with no kernels our own all vs one method actually gets the same accuracy and error rate as the
# Sklearn model of all vs one with no kernels. They both have a accuracy of around 88%

In [103]:
#Both multiclass without kernel and multiclass Sklearn without kearnel had 6 misclassifications

In [104]:
#Multiclass classification with kernels

In [105]:
#Polynomial

In [106]:
results = np.array([])

In [107]:
#CP vs all

In [108]:
def cpvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=45)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = (np.matmul(XPY,XPY.transpose())+0)**3

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [109]:
cpwherediff, cpwght, cpydual_test, cpb = cpvsall(X,Y)

In [110]:
print(len(cpwherediff)/len(cpydual_test))

0.0392156862745098


In [111]:
#creating fit on main test for cp

In [112]:
cpx_test = x_test[np.where(y_test == "cp")[0]]
cpy_test = y_test[np.where(y_test == "cp")[0]]

In [113]:
cpyfit = np.matmul(cpx_test,cpwght) + cpb
cpyfitsign = np.sign(cpyfit)
print(cpyfitsign)
print(cpy_test)

[1 1 1 1 1 1 -1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
['cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp']


In [114]:
results = np.append(results,cpyfitsign)

In [115]:
#im vs all

In [116]:
def imvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=45)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = (np.matmul(XPY,XPY.transpose())+0)**3

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [117]:
imwherediff, imwght, imydual_test, imb = imvsall(X,Y)

In [118]:
print(len(imwherediff)/len(imydual_test))

0.11764705882352941


In [119]:
#creating fit on main test for im

In [120]:
imx_test = x_test[np.where(y_test == "im")[0]]
imy_test = y_test[np.where(y_test == "im")[0]]

In [121]:
imyfit = np.matmul(imx_test,imwght) + imb
imyfitsign = np.sign(imyfit)
print(imyfitsign)
print(imy_test)

[1 1 -1 1 1 1 1 1]
['im' 'im' 'im' 'im' 'im' 'im' 'im' 'im']


In [122]:
results = np.append(results,imyfitsign)

In [123]:
#ms va all

In [124]:
def msvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=45)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = (np.matmul(XPY,XPY.transpose())+0)**3

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [125]:
mswherediff, mswght, msydual_test, msb = msvsall(X,Y)

In [126]:
print(len(mswherediff)/len(msydual_test))

0.0


In [127]:
#creating fit on main test for ms

In [128]:
msx_test = x_test[np.where(y_test == "mS")[0]]
msy_test = y_test[np.where(y_test == "mS")[0]]

In [129]:
msyfit = np.matmul(msx_test,mswght) + msb
msyfitsign = np.sign(msyfit)
print(msyfitsign)
print(msy_test)

[]
[]


In [130]:
results = np.append(results,msyfitsign)

In [131]:
#ml vs all

In [132]:
def mlvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=45)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = (np.matmul(XPY,XPY.transpose())+0)**3

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [133]:
mlwherediff, mlwght, mlydual_test, mlb = mlvsall(X,Y)

In [134]:
print(len(mlwherediff)/len(mlydual_test))

0.0392156862745098


In [135]:
#creating fit on main test for ml

In [136]:
mlx_test = x_test[np.where(y_test == "mL")[0]]
mly_test = y_test[np.where(y_test == "mL")[0]]

In [137]:
mlyfit = np.matmul(mlx_test,mlwght) + mlb
mlyfitsign = np.sign(mlyfit)
print(mlyfitsign)
print(mly_test)

[-1]
['mL']


In [138]:
results = np.append(results,mlyfitsign)

In [139]:
#mu vs all

In [140]:
def muvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=45)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = (np.matmul(XPY,XPY.transpose())+ 0)**3

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [141]:
muwherediff, muwght, muydual_test, mub = muvsall(X,Y)

In [142]:
print(len(muwherediff)/len(muydual_test))

0.0784313725490196


In [143]:
#creating fit on main test for mu

In [144]:
mux_test = x_test[np.where(y_test == "mU")[0]]
muy_test = y_test[np.where(y_test == "mU")[0]]

In [145]:
muyfit = np.matmul(mux_test,muwght) + mub
muyfitsign = np.sign(muyfit)
print(muyfitsign)
print(muy_test)

[-1 1 -1 1 1]
['mU' 'mU' 'mU' 'mU' 'mU']


In [146]:
results = np.append(results,muyfitsign)

In [147]:
#om vs all

In [148]:
def omvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=45)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = (np.matmul(XPY,XPY.transpose())+0)**3

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [149]:
omwherediff, omwght, omydual_test, omb = omvsall(X,Y)

In [150]:
print(len(omwherediff)/len(omydual_test))

0.0392156862745098


In [151]:
#creating fit on main test for om

In [152]:
omx_test = x_test[np.where(y_test == "om")[0]]
omy_test = y_test[np.where(y_test == "om")[0]]

In [153]:
omyfit = np.matmul(omx_test,omwght) + omb
omyfitsign = np.sign(omyfit)
print(omyfitsign)
print(omy_test)

[-1 1]
['om' 'om']


In [154]:
results = np.append(results,omyfitsign)

In [155]:
# pp vs all

In [156]:
def ppvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=45)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = (np.matmul(XPY,XPY.transpose())+0)**3

    YM = np.outer(ydual_train[1:],ydual_train[1:])
    AY = np.outer(A[0,1:],ydual_train[1:])
    YA = np.outer(ydual_train[1:],A[0,1:])


    Y0S = ydual_train[0]**2
    M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = np.diagonal(A)
    t1    = A[0,0]
    t2    = 2*np.multiply(A[0,:],ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [157]:
ppwherediff, ppwght, ppydual_test, ppb= ppvsall(X,Y)

In [158]:
print(len(ppwherediff)/len(ppydual_test))

0.11764705882352941


In [159]:
#creating fit on main test for pp

In [160]:
ppx_test = x_test[np.where(y_test == "pp")[0]]
ppy_test = y_test[np.where(y_test == "pp")[0]]

In [161]:
ppyfit = np.matmul(ppx_test,ppwght) + ppb
ppyfitsign = np.sign(ppyfit)
print(ppyfitsign)
print(ppy_test)

[1 1 1 1 1 1]
['pp' 'pp' 'pp' 'pp' 'pp' 'pp']


In [162]:
results = np.append(results,ppyfitsign)
results.size

51

In [163]:
Right = results[results == 1].size
Wrong = results[results == -1].size
Total = results.size
print("Accuracy = ", Right/Total)
print("Error = ", Wrong/Total)

Accuracy =  0.8823529411764706
Error =  0.11764705882352941


In [164]:
# Sklearn for mulitclass radial kernel

In [228]:
model = SVC(kernel = "poly", degree = 3)
m = OneVsRestClassifier(model)
m.fit(x_train,y_train)

OneVsRestClassifier(estimator=SVC(kernel='poly'))

In [229]:
print("Accuracy = ", m.score(x_test,y_test) )
print("Error = ", 1 - m.score(x_test,y_test))

Accuracy =  0.9215686274509803
Error =  0.07843137254901966


In [ ]:
# It looks like are polynomial kernel multiclass classification and the SKlearn polynomial kernel classifier have
# very similar Accuracy levels of 0.88 for our own version and 0.92 for the sklearn version.

In [165]:
#Radial

In [166]:
results = np.array([])

In [167]:
#CP vs all

In [168]:
def cpvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=44)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.exp(-(LA.norm(XPY-XPY)**2)/np.var(XPY))

    #YM = np.outer(ydual_train[1:],ydual_train[1:])
    #AY = np.outer(A[0,1:],ydual_train[1:])
    #YA = np.outer(ydual_train[1:],A[0,1:])


    #Y0S = ydual_train[0]**2
    #M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = A
    t1    = A
    t2    = 2*np.multiply(A,ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [169]:
cpwherediff, cpwght, cpydual_test, cpb = cpvsall(X,Y)

In [170]:
print(len(cpwherediff)/len(cpydual_test))

0.1568627450980392


In [171]:
#creating fit on main test for cp

In [172]:
cpx_test = x_test[np.where(y_test == "cp")[0]]
cpy_test = y_test[np.where(y_test == "cp")[0]]

In [173]:
cpyfit = np.matmul(cpx_test,cpwght) + cpb
cpyfitsign = np.sign(cpyfit)
print(cpyfitsign)
print(cpy_test)

[1 1 1 -1 1 1 -1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 -1]
['cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp']


In [174]:
results = np.append(results,cpyfitsign)

In [175]:
#im vs all

In [176]:
def imvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=44)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.exp(-(LA.norm(XPY-XPY)**2)/np.var(XPY))

    #YM = np.outer(ydual_train[1:],ydual_train[1:])
    #AY = np.outer(A[0,1:],ydual_train[1:])
    #YA = np.outer(ydual_train[1:],A[0,1:])


    #Y0S = ydual_train[0]**2
    #M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = A
    t1    = A
    t2    = 2*np.multiply(A,ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [177]:
imwherediff, imwght, imydual_test, imb = imvsall(X,Y)

In [178]:
print(len(imwherediff)/len(imydual_test))

0.2549019607843137


In [179]:
#creating fit on main test for im

In [180]:
imx_test = x_test[np.where(y_test == "im")[0]]
imy_test = y_test[np.where(y_test == "im")[0]]

In [181]:
imyfit = np.matmul(imx_test,imwght) + imb
imyfitsign = np.sign(imyfit)
print(imyfitsign)
print(imy_test)

[1 1 1 1 -1 1 1 1]
['im' 'im' 'im' 'im' 'im' 'im' 'im' 'im']


In [182]:
results = np.append(results,imyfitsign)

In [183]:
#ms va all

In [184]:
def msvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=44)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.exp(-(LA.norm(XPY-XPY)**2)/np.var(XPY))

    #YM = np.outer(ydual_train[1:],ydual_train[1:])
    #AY = np.outer(A[0,1:],ydual_train[1:])
    #YA = np.outer(ydual_train[1:],A[0,1:])


    #Y0S = ydual_train[0]**2
    #M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = A
    t1    = A
    t2    = 2*np.multiply(A,ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [185]:
mswherediff, mswght, msydual_test, msb = msvsall(X,Y)

In [186]:
print(len(mswherediff)/len(msydual_test))

0.0196078431372549


In [187]:
#creating fit on main test for ms

In [188]:
msx_test = x_test[np.where(y_test == "mS")[0]]
msy_test = y_test[np.where(y_test == "mS")[0]]

In [189]:
msyfit = np.matmul(msx_test,mswght) + msb
msyfitsign = np.sign(msyfit)
print(msyfitsign)
print(msy_test)

[]
[]


In [190]:
results = np.append(results,msyfitsign)

In [191]:
#ml vs all

In [192]:
def mlvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=44)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.exp(-(LA.norm(XPY-XPY)**2)/np.var(XPY))
    
    #YM = np.outer(ydual_train[1:],ydual_train[1:])
    #AY = np.outer(A[0,1:],ydual_train[1:])
    #YA = np.outer(ydual_train[1:],A[0,1:])


    #Y0S = ydual_train[0]**2
    #M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = A
    t1    = A
    t2    = 2*np.multiply(A,ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [193]:
mlwherediff, mlwght, mlydual_test, mlb = mlvsall(X,Y)

In [194]:
print(len(mlwherediff)/len(mlydual_test))

0.0392156862745098


In [195]:
#creating fit on main test for ml

In [196]:
mlx_test = x_test[np.where(y_test == "mL")[0]]
mly_test = y_test[np.where(y_test == "mL")[0]]

In [197]:
mlyfit = np.matmul(mlx_test,mlwght) + mlb
mlyfitsign = np.sign(mlyfit)
print(mlyfitsign)
print(mly_test)

[-1]
['mL']


In [198]:
results = np.append(results,mlyfitsign)

In [199]:
#mu vs all

In [200]:
def muvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereisom] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=44)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.exp(-(LA.norm(XPY-XPY)**2)/np.var(XPY))

    #YM = np.outer(ydual_train[1:],ydual_train[1:])
    #AY = np.outer(A[0,1:],ydual_train[1:])
    #YA = np.outer(ydual_train[1:],A[0,1:])


    #Y0S = ydual_train[0]**2
    #M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = A
    t1    = A
    t2    = 2*np.multiply(A,ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [201]:
muwherediff, muwght, muydual_test, mub = muvsall(X,Y)

In [202]:
print(len(muwherediff)/len(muydual_test))
print(muydual_test)
print(y_test)

0.2549019607843137
[-1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1.  1. -1. -1. -1. -1. -1. -1.  1. -1. -1.  1. -1. -1. -1. -1.
  1. -1. -1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1.]
['mL' 'cp' 'mU' 'cp' 'cp' 'mU' 'cp' 'pp' 'cp' 'pp' 'pp' 'cp' 'im' 'cp'
 'cp' 'pp' 'cp' 'cp' 'im' 'cp' 'mU' 'om' 'cp' 'cp' 'cp' 'im' 'cp' 'im'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'mU' 'cp' 'om' 'mU' 'cp' 'pp' 'cp' 'cp'
 'cp' 'pp' 'cp' 'im' 'im' 'im' 'cp' 'cp' 'im']


In [203]:
#creating fit on main test for mu

In [204]:
mux_test = x_test[np.where(y_test == "mU")[0]]
muy_test = y_test[np.where(y_test == "mU")[0]]

In [205]:
muyfit = np.matmul(mux_test,muwght) + mub
muyfitsign = np.sign(muyfit)
print(muyfitsign)
print(muy_test)

[1 1 1 1 1]
['mU' 'mU' 'mU' 'mU' 'mU']


In [206]:
results = np.append(results,muyfitsign)

In [207]:
#om vs all

In [208]:
def omvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereispp] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=44)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.exp(-(LA.norm(XPY-XPY)**2)/np.var(XPY))

    #YM = np.outer(ydual_train[1:],ydual_train[1:])
    #AY = np.outer(A[0,1:],ydual_train[1:])
    #YA = np.outer(ydual_train[1:],A[0,1:])


    #Y0S = ydual_train[0]**2
    #M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = A
    t1    = A
    t2    = 2*np.multiply(A,ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [209]:
omwherediff, omwght, omydual_test, omb = omvsall(X,Y)

In [210]:
print(len(omwherediff)/len(omydual_test))

0.09803921568627451


In [211]:
#creating fit on main test for om

In [212]:
omx_test = x_test[np.where(y_test == "om")[0]]
omy_test = y_test[np.where(y_test == "om")[0]]

In [213]:
omyfit = np.matmul(omx_test,omwght) + omb
omyfitsign = np.sign(omyfit)
print(omyfitsign)
print(omy_test)

[1 1]
['om' 'om']


In [214]:
results = np.append(results,omyfitsign)

In [215]:
# pp vs all

In [216]:
def ppvsall(X,Y):
    #Creating data
    cp = 'cp'
    im = 'im'
    ms = "mS"
    ml = "mL"
    mu = "mU"
    om = "om"
    pp = "pp"


    whereiscp = np.where(Y==cp)[0]
    whereisim = np.where(Y==im)[0]
    whereisms = np.where(Y==ms)[0]
    whereisml = np.where(Y==ml)[0]
    whereismu = np.where(Y==mu)[0]
    whereisom = np.where(Y==om)[0]
    whereispp = np.where(Y==pp)[0]

    indxall = np.concatenate((whereiscp,whereisim,whereisms,whereisml,whereismu,whereisom,whereispp))
    
    Xall = X[indxall]
    Yall = np.ones(len(indxall))
    Yall[whereiscp] =-1
    Yall[whereisim] =-1
    Yall[whereisms] =-1
    Yall[whereisml] =-1
    Yall[whereismu] =-1
    Yall[whereisom] =-1

    xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(Xall, Yall, test_size = 0.15, random_state=44)
    
    #Creating Classifier
    N = len(ydual_train)
    XPY = xdual_train.copy()
    for i in range(N):
       if ydual_train[i]==-1:
          XPY[i,:] =-1 * XPY[i,:]   

    A = np.exp(-(LA.norm(XPY-XPY)**2)/np.var(XPY))

    #YM = np.outer(ydual_train[1:],ydual_train[1:])
    #AY = np.outer(A[0,1:],ydual_train[1:])
    #YA = np.outer(ydual_train[1:],A[0,1:])


    #Y0S = ydual_train[0]**2
    #M = A[1:,1:] + A[0,0]*YM/Y0S - AY/ydual_train[0] - YA/ydual_train[0]
    
    aw = np.zeros(N)

    bright = np.zeros(N)
    bright = 1 - ydual_train/ydual_train[0]

    adiag = A
    t1    = A
    t2    = 2*np.multiply(A,ydual_train)/ydual_train[0]
    t3    = adiag + t1 - t2
    indxnzero = np.where(t3 != 0)
    aw[indxnzero[0]]    = np.divide(bright[indxnzero],t3[indxnzero])
    aw[0] =-sum(ydual_train[indxnzero]*aw[indxnzero])/ydual_train[0]
    
    YA = np.zeros(N)
    YA[0] = ydual_train[0]*aw[0]
    YA[indxnzero] = ydual_train[indxnzero]*aw[indxnzero]
    wght = sum(xdual_train * YA[:,None])
    b =sum(ydual_train - np.matmul(xdual_train,wght))/N
    
    #Creating fit
    yfit = np.matmul(xdual_test,wght) + b
    yfitsign = np.sign(yfit)
    ydiff = yfitsign - np.asfarray(ydual_test)
    wherediff = np.where(ydiff != 0)
    
    return wherediff[0], wght, ydual_test, b

In [217]:
ppwherediff, ppwght, ppydual_test, ppb= ppvsall(X,Y)

In [218]:
print(len(ppwherediff)/len(ppydual_test))

0.17647058823529413


In [219]:
#creating fit on main test for pp

In [220]:
ppx_test = x_test[np.where(y_test == "pp")[0]]
ppy_test = y_test[np.where(y_test == "pp")[0]]

In [221]:
ppyfit = np.matmul(ppx_test,ppwght) + ppb
ppyfitsign = np.sign(ppyfit)
print(ppyfitsign)
print(ppy_test)

[1 1 1 1 1 1]
['pp' 'pp' 'pp' 'pp' 'pp' 'pp']


In [222]:
results = np.append(results,ppyfitsign)
results.size

51

In [223]:
Right = results[results == 1].size
Wrong = results[results == -1].size
Total = results.size
print("Accuracy = ", Right/Total)
print("Error = ", Wrong/Total)

Accuracy =  0.9019607843137255
Error =  0.09803921568627451


In [224]:
# Sklearn for mulitclass radial kernel

In [225]:
model = SVC(kernel = "rbf")
m = OneVsRestClassifier(model)
m.fit(x_train,y_train)

OneVsRestClassifier(estimator=SVC())

In [226]:
print("Accuracy = ", m.score(x_test,y_test) )
print("Error = ", 1 - m.score(x_test,y_test))

Accuracy =  0.9215686274509803
Error =  0.07843137254901966


In [227]:
# It looks like are radial kernel multiclass classification and the SKlearn radial kernel classifier have
# very similar Accuracy levels of 0.90 for our own version and 0.92 for the sklearn version.